In [4]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import dotenv_values
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.ai.vision.imageanalysis.aio import ImageAnalysisClient
from azure.storage.blob import BlobServiceClient

In [5]:
config = dotenv_values(".env")
open_ai_key=config["open_ai_key"]
open_ai_endpoint=config["open_ai_endpoint"]
open_ai_endpoint_2=config["open_ai_endpoint_2"]
vision_endpoint=config["open_ai_endpoint_3"]
connect_str=config["connection_str"]

In [6]:
connection_string = config["connection_string"]
container_name = config["container_name"]
blob_service_client = BlobServiceClient.from_connection_string(connect_str)


container_client = blob_service_client.get_container_client(container_name)

urls = []
for blob in container_client.list_blobs():
    blob_client = container_client.get_blob_client(blob.name)
    urls.append(blob_client.url)
    print(blob_client.url)


https://tcctocrpoc.blob.core.windows.net/tcct-ocr-poc/17386590297053997295044438274399%20-%20Victor%20Lee.jpg
https://tcctocrpoc.blob.core.windows.net/tcct-ocr-poc/17386590966462230082736051626241%20-%20Victor%20Lee.jpg
https://tcctocrpoc.blob.core.windows.net/tcct-ocr-poc/17387186556135444632046881269223%20-%20Victor%20Lee.jpg
https://tcctocrpoc.blob.core.windows.net/tcct-ocr-poc/17387375666977370569976893425821%20-%20Victor%20Lee.jpg
https://tcctocrpoc.blob.core.windows.net/tcct-ocr-poc/1738828911937654016708644032092%20-%20Victor%20Lee.jpg
https://tcctocrpoc.blob.core.windows.net/tcct-ocr-poc/17388290984957689282649041447964%20-%20Victor%20Lee.jpg
https://tcctocrpoc.blob.core.windows.net/tcct-ocr-poc/1738902074708779141148581966324%20-%20Victor%20Lee.jpg
https://tcctocrpoc.blob.core.windows.net/tcct-ocr-poc/17391657755742514674626984958435%20-%20Victor%20Lee.jpg
https://tcctocrpoc.blob.core.windows.net/tcct-ocr-poc/1739165892913360950025615499298%20-%20Victor%20Lee.jpg
https://tccto

In [7]:
def get_blob_sas_urls(container_client, blob_names):
    urls = []
    for blob_name in blob_names:
        blob_client = container_client.get_blob_client(blob_name)
        
        from datetime import datetime, timedelta
        from azure.storage.blob import generate_blob_sas, BlobSasPermissions
        
        sas_token = generate_blob_sas(
            account_name=blob_service_client.account_name,
            container_name=container_name,
            blob_name=blob_name,
            account_key=blob_service_client.credential.account_key,
            permission=BlobSasPermissions(read=True),
            expiry=datetime.utcnow() + timedelta(hours=1)
        )
        
        sas_url = f"{blob_client.url}?{sas_token}"
        urls.append(sas_url)
    
    return urls

In [8]:
blob_names = [blob.name for blob in container_client.list_blobs()]

In [9]:
urls = get_blob_sas_urls(container_client, blob_names)

C:\Users\phuvit.j\AppData\Local\Temp\ipykernel_31244\2271987532.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  expiry=datetime.utcnow() + timedelta(hours=1)


In [10]:
model = ChatCompletionsClient(
    endpoint=open_ai_endpoint,
    credential=AzureKeyCredential(open_ai_key),
)

In [11]:
response = model.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="Explain Riemann's conjecture in 1 paragraph"),
    ],
    model="gpt-4o-mini"
)

print(response.choices[0].message.content)

Riemann's conjecture, also known as the Riemann Hypothesis, is a famous unsolved problem in mathematics that asserts that all non-trivial zeros of the Riemann zeta function, defined as ζ(s) for complex numbers s, lie on the critical line where the real part of s is equal to 1/2. Formulated by the German mathematician Bernhard Riemann in 1859, this conjecture has profound implications for number theory, particularly in the distribution of prime numbers. The zeta function is deeply connected to the distribution of primes through its relationship with their density, and the hypothesis is central to understanding the patterns and properties of prime numbers. Despite extensive numerical evidence supporting the conjecture, no formal proof has yet been established, making it one of the seven "Millennium Prize Problems" that carries a reward of one million dollars for a solution.


In [16]:
async def sample_ocr_image_url_async(url):
    # Create an asynchronous Image Analysis client
    client = ImageAnalysisClient(
    endpoint=vision_endpoint,
    credential=AzureKeyCredential(open_ai_key)
)

    # Extract text (OCR) from an image URL, asynchronously.
    result = await client.analyze_from_url(
        image_url=url,
        visual_features=[VisualFeatures.READ],
        
    )

    await client.close()

    response = await model.complete(
        messages=[
            SystemMessage(content="You are a helpful assistant."),
            UserMessage(content=f"From this json, please extract this receipt json to get only the important feature which is each receipt line in a json format: {result}"),
        ],
        model="gpt-4o-mini"
    )
    
    await model.close()
    
    print(response)
    
    return
    # Print text (OCR) analysis results to the console
    # print("Image analysis results:")
    # print(" Read:")
    # if result.read is not None:
    #     for line in result.read.blocks[0].lines:
    #         print(f"   Line: '{line.text}', Bounding box {line.bounding_polygon}")
    #         for word in line.words:
    #             print(f"     Word: '{word.text}', Bounding polygon {word.bounding_polygon}, Confidence {word.confidence:.4f}")
    # print(f" Image height: {result.metadata.height}")
    # print(f" Image width: {result.metadata.width}")
    # print(f" Model version: {result.model_version}")

In [17]:
# for url in urls:
await sample_ocr_image_url_async(urls[0])

HttpResponseError: (429) Requests to the Creates a model response for the given chat conversation. Operation under Azure AI Model Inference API version 2024-05-01-preview have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 60 seconds. Please contact Azure support service if you would like to further increase the default rate limit.
Code: 429
Message: Requests to the Creates a model response for the given chat conversation. Operation under Azure AI Model Inference API version 2024-05-01-preview have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 60 seconds. Please contact Azure support service if you would like to further increase the default rate limit.